### **Financial Network Fraud Detection in Graphs**
#### **Dataset preprocessing**

The notebook provides a comprehensive data preprocessing and feature extraction for a fraud detection task on a private transaction dataset. It includes the steps to clean, transform, handling missing values, and engineer features from raw data to prepare it for Graph construction and ML models.  

It includes the following key components:  
1. **Data preparation**:  
- Loading raw data files
- Filters, merges, selects, and aggregates data

2. **Data cleaning & transformation**:
- Handling missing values
- Data type conversion
- Handling duplicates
- Temporary data transformation
- Handling categorical & numerical data
- Text data cleaning

3. **Feature extraction & engineering based on domain knowledge**:
- Adding new derived features
- Computing time-based features
- Adding relative time features

4. **Splitting data**:
- Splitting data into training, validation, and testing sets

Author: Amr Sherif  
Created Date: 2024-11-05  
Updated Date: 2024-11-08  
Version: 1.0

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [2]:
cwd = Path.cwd()
rawData_dir = cwd / "rawData"

In [3]:
orders = pd.read_csv(rawData_dir / 'orders.csv')
installments = pd.read_csv(rawData_dir / 'installments.csv')
installments_plans = pd.read_csv(rawData_dir / 'installments_plans.csv')
transactions = pd.read_csv(rawData_dir / 'transactions.csv')

addresses = pd.read_csv(rawData_dir / 'addresses.csv')
cart = pd.read_csv(rawData_dir / 'cart.csv')
checkouts = pd.read_csv(rawData_dir / 'checkouts.csv')
customers = pd.read_csv(rawData_dir / 'customers.csv')
idv = pd.read_csv(rawData_dir / 'idv.csv')
ekata = pd.read_csv(rawData_dir / 'ekata.csv')
merchants = pd.read_csv(rawData_dir / 'merchants.csv')

seon = pd.read_csv(rawData_dir / 'seon.csv')
orders_district = pd.read_csv(rawData_dir / 'orders_district.csv')
bin_codes = pd.read_csv(rawData_dir / 'bin_codes.csv')
merchants_category = pd.read_csv(rawData_dir / 'merchants_category.csv')
orders_fraud_category = pd.read_csv(rawData_dir / 'orders_fraud_category.csv')

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/441442133.py:1: DtypeWarning: Columns (2,25,26,27,28,29,35,37,39,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv(rawData_dir / 'orders.csv')
/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/441442133.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  installments_plans = pd.read_csv(rawData_dir / 'installments_plans.csv')
/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/441442133.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  cart = pd.read_csv(rawData_dir / 'cart.csv')
/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/441442133.py:9: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  customers = pd.read_csv(rawData_dir / 'customers.csv')
/var/f

In [4]:
orders = orders.loc[(orders['num_instalments'] >1) & 
                    (orders['currency'] == 'AED') & 
                    (orders['status'].isin(['captured', 'denied']))
                    ]

installments_plans = installments_plans.loc[(installments_plans['num_instalments'] >1) &
                                            (installments_plans['currency'] == 'AED')
                                            ]

checkouts = checkouts.loc[(checkouts['num_instalments'] >1)]


In [5]:
# add suffix to columns
orders = orders.add_suffix('_orders')
installments_plans = installments_plans.add_suffix('_installments_plans')
checkouts = checkouts.add_suffix('_checkouts')
addresses = addresses.add_suffix('_addresses')
customers = customers.add_suffix('_customers')
cart = cart.add_suffix('_cart')
idv = idv.add_suffix('_idv')
ekata = ekata.add_suffix('_ekata')
merchants = merchants.add_suffix('_merchants')
seon = seon.add_suffix('_seon')
orders_district = orders_district.add_suffix('_district')
bin_codes = bin_codes.add_suffix('_bin')
merchants_category = merchants_category.add_suffix('_merchantcategory')
orders_fraud_category = orders_fraud_category.add_suffix('_fraudcategory')

In [6]:
# eliminating irrelevant records for gift card & pp card >> ex. MAF
orders = orders.loc[~orders['checkout_type_orders'].isin(['gift-card', 'postpay-card'])]
orders = orders.loc[~(orders['customer_id_orders'] == '')]

# dropping duplicates and keeping the latest records in seon and ekata
seon = seon.sort_values(by='created_at_seon', ascending=False)
seon = seon.drop_duplicates(subset='decision_id_seon', keep='first')

ekata = ekata.sort_values(by='created_at_ekata', ascending=False)
ekata = ekata.drop_duplicates(subset='decision_id_ekata', keep='first')

In [7]:
# PK & FK data type conversion
orders['order_id_orders'] = orders['order_id_orders'].astype('int64')
installments_plans['order_id_installments_plans'] = installments_plans['order_id_installments_plans'].astype('int64')
checkouts['order_id_checkouts'] = checkouts['order_id_checkouts'].astype('int64')
orders['shipping_address_id_orders'] = orders['shipping_address_id_orders'].astype('Int64')
addresses['id_addresses'] = addresses['id_addresses'].astype('int64')
customers['customer_id_customers'] = customers['customer_id_customers'].astype('int64')
orders['customer_id_orders'] = orders['customer_id_orders'].astype('int64')
cart['order_id_cart'] = cart['order_id_cart'].astype('Int64')
idv['customer_id_idv'] = idv['customer_id_idv'].astype('int64')
orders_district['order_id_district'] = orders_district['order_id_district'].astype('int64')
merchants_category['merchant_id_merchantcategory'] = merchants_category['merchant_id_merchantcategory'].astype('str')
orders['merchant_id_orders'] = orders['merchant_id_orders'].astype('str')

In [8]:
# orders['shipping_address_id_orders'].dtype
# addresses['id_addresses'].dtype
# cart['order_id_cart'].dtype
# idv['customer_id_idv'].dtype
# ekata['decision_id_ekata'].dtype
# orders['decision_id_orders'].dtype
# seon['decision_id_seon'].dtype
# orders_district['order_id_district'].dtype
# bin_codes['bin_bin'].dtype
# orders['card_first_six_digits_orders'].dtype
# merchants_category['merchant_id_merchantcategory'].dtype
# orders['merchant_id_orders'].dtype
# orders_fraud_category['order_id_fraudcategory'].dtype
# orders['decision_id_orders'].dtype
# installments_plans['card_first_six_digits_installments_plans'].dtype

In [9]:
orders_agg = pd.merge(orders, installments_plans, left_on='order_id_orders', right_on='order_id_installments_plans', how='left')
orders_agg = pd.merge(orders_agg, checkouts, left_on='order_id_orders', right_on='order_id_checkouts', how='left')
orders_agg = pd.merge(orders_agg, addresses, left_on='shipping_address_id_orders', right_on='id_addresses', how='left')

In [33]:
cart = cart[['order_id_cart', 'name_cart', 'qty_cart']].reset_index()
cart = cart.dropna()

In [34]:
def aggregate_order(df):
    max_qty_row = df.loc[df['qty_cart'].idxmax()]
    
    max_qty = max_qty_row['qty_cart']
    max_qty_name = max_qty_row['name_cart']
    concatenated_names = ', '.join(df['name_cart'])
    total_qty = df['qty_cart'].sum()

    return pd.Series({'concatenated_names': concatenated_names,
                      'total_quantity': total_qty,
                      'max_quantity_name': max_qty_name,
                      'max_quantity': max_qty})

In [35]:
cart_agg = cart.groupby('order_id_cart').apply(aggregate_order).reset_index()

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/963531719.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cart_agg = cart.groupby('order_id_cart').apply(aggregate_order).reset_index()


In [36]:
orders_agg = pd.merge(orders_agg, cart_agg, left_on='order_id_orders', right_on='order_id_cart', how='left')

In [37]:
orders_agg = pd.merge(orders_agg, customers, left_on='customer_id_orders', right_on='customer_id_customers', how='left')

In [38]:
orders_agg = pd.merge(orders_agg, idv, left_on='customer_id_orders', right_on='customer_id_idv', how='left')

In [39]:
orders_agg = pd.merge(orders_agg, ekata, left_on='decision_id_orders', right_on='decision_id_ekata', how='left')
orders_agg = pd.merge(orders_agg, seon, left_on='decision_id_orders', right_on='decision_id_seon', how='left')
orders_agg = pd.merge(orders_agg, orders_district, left_on='order_id_orders', right_on='order_id_district', how='left')
orders_agg = pd.merge(orders_agg, bin_codes, left_on='card_first_six_digits_orders', right_on='bin_bin', how='left')
orders_agg = pd.merge(orders_agg, merchants_category, left_on='merchant_id_orders', right_on='merchant_id_merchantcategory', how='left')
orders_agg = pd.merge(orders_agg, orders_fraud_category, left_on='order_id_orders', right_on='order_id_fraudcategory', how='left')

In [40]:
# orders_agg[orders_agg['order_id_orders'].duplicated()==True]

,order_id_orders,shipping_id_orders,merchant_id_orders,total_amount_orders,status_orders,status_changed_orders,billing_address_id_orders,convenience_fee_orders,tax_amount_orders,created_orders,...,bin_bin,card_type_cleaned_bin,card_brand_cleaned_bin,card_level_cleaned_bin,merchant_id_merchantcategory,merchant_name_merchantcategory,category_merchantcategory,order_id_fraudcategory,fraud_category_fraudcategory,status_fraudcategory


In [127]:
# select desired columns for visibility
# suffix = 'installments_plans'
# orders_agg.filter(regex=f'.*_{suffix}$')

In [42]:
# filter and separate captured orders (only captured non-fraud) and all fraud orders (captured + denied)
orders_agg_fraud = orders_agg[orders_agg['fraud_category_fraudcategory'].isin(['Fraud', 'Suspected Fraud'])].reset_index(drop=True)

orders_agg_captured = orders_agg[orders_agg['status_orders'].isin(['captured'])].reset_index(drop=True)

orders_agg_captured = orders_agg_captured[~orders_agg_captured['fraud_category_fraudcategory'].isin(['Fraud', 'Suspected Fraud'])].reset_index(drop=True)
orders_agg_captured = orders_agg_captured.sort_values(by='created_orders', ascending=False)

In [43]:
orders_agg_captured.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376049 entries, 264693 to 105355
Columns: 375 entries, order_id_orders to status_fraudcategory
dtypes: Int64(2), float64(102), int64(3), object(268)
memory usage: 1.1+ GB


In [ ]:
# select a subset of captured orders for initial experimentation
# orders_agg_captured = orders_agg_captured.iloc[:105000]

In [44]:
orders_agg_captured = pd.concat([orders_agg_captured, orders_agg_fraud], axis=0, ignore_index=True)
orders_agg_captured = orders_agg_captured.sample(frac=1).reset_index(drop=True)

In [45]:
orders_agg_cleaned = orders_agg_captured[[
                        # installments_plans
                        'payment_method_brand_installments_plans', 'payment_method_expires_installments_plans', 'card_last_four_digits_installments_plans', 'card_first_six_digits_installments_plans', 
    'is_apple_pay_installments_plans',
                        'payment_method_country_installments_plans', 'payment_method_bank_installments_plans', 'payment_method_type_installments_plans', 
    'customer_email_installments_plans', 
    # 'secure_installments_plans', 
                        'payment_method_fingerprint_installments_plans', 'id_number_installments_plans',
                        # orders
                        'order_id_orders', 'total_amount_orders', 'created_orders', 'customer_email_orders', 'customer_date_joined_orders', 
                       'merchant_name_orders', 'category_merchantcategory', 'checkout_type_orders', 'num_instalments_orders', 'ip_address_orders', 'user_agent_orders', 
                       # 'secure_orders', 
    'device_fingerprint_orders', 'checkout_verified_orders', 'checkout_metadata_orders', 
                       'payment_method_fingerprint_orders', 'card_last_four_digits_orders', 'payment_method_expires_orders', 
                       'card_first_six_digits_orders', 'is_apple_pay_orders', 'payment_method_country_orders','payment_method_bank_orders', 
                       # bin codes
                       'card_type_cleaned_bin', 'card_brand_cleaned_bin', 'card_level_cleaned_bin', 'date_of_birth_installments_plans', 
                       'phone_installments_plans', 
                        # addresses details
                        'first_name_addresses', 'last_name_addresses', 'line1_addresses', 'line2_addresses', 'final_address_district', 
                       'formatted_address_district', 'state_emirate_district', 'district_district', 'country_district', 'phone_addresses', 
                        # cart
                        'concatenated_names', 'total_quantity', 'max_quantity_name', 'max_quantity', 
                        # customers table
                        'first_name_customers', 'last_name_customers', 'id_number_customers', 'email_customers', 
                       'date_of_birth_customers', 'phone_customers', 
                        # idv
                        'full_name_idv', 'identity_number_idv', 'date_of_birth_formatted_idv', 'date_of_expiry_formatted_idv', 
                       'place_of_issue_idv', 'sex_idv', 'employer_idv', 'occupation_group_idv', 'nationality_code_idv', 
                        # ekata
                        'ip_last_seen_days_ekata', 'primary_email_to_name_ekata', 
                        'ip_geolocation_country_code_ekata', 
    # 'primary_phone_valid_ekata', 
    'primary_address_validity_level_ekata', 
                       'primary_email_domain_creation_date_ekata', 'ip_risk_ekata', 'ip_risk_score_ekata', 'primary_email_last_seen_days_ekata', 'primary_email_valid_ekata', 
                       'primary_phone_to_name_ekata', 'ip_primary_address_distance_ekata', 'primary_phone_carrier_ekata', 'identity_network_score_ekata', 'identity_risk_score_ekata', 
                       'primary_phone_line_type_ekata', 'primary_address_first_seen_days_ekata', 'primary_phone_ip_last_seen_days_ekata', 'ip_connection_type_ekata', 
                       'primary_address_to_name_ekata', 'primary_email_first_seen_days_ekata', 
                        # seon
                        # 'phone_details_valid_seon', 
    'phone_details_score_seon', 'phone_details_country_seon', 
                       'phone_details_carrier_seon', 'phone_details_account_details_whatsapp_registered_seon', 'phone_details_account_details_twitter_registered_seon', 
                       # 'phone_details_account_details_yahoo_registered_seon', 
    'phone_details_account_details_facebook_registered_seon', 'phone_details_account_details_telegram_registered_seon', 
                       'phone_details_account_details_instagram_registered_seon', 'phone_details_account_details_google_registered_seon', 
                       # 'phone_details_type_seon', 
    'state_seon', 'ip_details_country_seon', 'ip_details_public_proxy_seon', 'ip_details_city_seon', 
    # 'ip_details_web_proxy_seon', 
    'ip_details_spam_number_seon', 
                       'ip_details_type_seon', 'ip_details_isp_name_seon', 'ip_details_score_seon', 'ip_details_vpn_seon', 
    # 'ip_details_tor_seon', 
    'ip_details_state_prov_seon', 'fraud_score_seon', 
                       'email_details_account_details_github_registered_seon', 'email_details_account_details_facebook_registered_seon', 'email_details_account_details_spotify_registered_seon',
                       'email_details_account_details_google_registered_seon', 'email_details_account_details_pinterest_registered_seon', 'email_details_account_details_instagram_registered_seon',
                       'email_details_account_details_linkedin_registered_seon', 'email_details_account_details_microsoft_registered_seon', 'email_details_account_details_apple_registered_seon', 
                       'email_details_account_details_twitter_registered_seon', 'email_details_deliverable_seon', 'email_details_score_seon', 'email_details_domain_details_created_seon', 
                       'email_details_domain_details_suspicious_tld_seon', 'email_details_domain_details_custom_seon', 'email_details_domain_details_valid_mx_seon', 
                       'email_details_domain_details_accept_all_seon', 'email_details_domain_details_registrar_name_seon', 'email_details_domain_details_registered_seon', 'email_details_domain_details_registered_to_seon', 
                       'email_details_domain_details_tld_seon', 'email_details_domain_details_disposable_seon', 'email_details_domain_details_spf_strict_seon', 'email_details_domain_details_website_exists_seon', 'email_details_domain_details_domain_seon',
                       'email_details_domain_details_dmarc_enforced_seon', 'email_details_breach_details_first_breach_seon', 'email_details_breach_details_haveibeenpwned_listed_seon', 'email_details_breach_details_number_of_breaches_seon',
                        # target
                       'fraud_category_fraudcategory']]

In [126]:
# orders_agg_cleaned.describe(include='all')

In [47]:
orders_agg_cleaned[orders_agg_cleaned['total_amount_orders'].isna()==True]

,payment_method_brand_installments_plans,payment_method_expires_installments_plans,card_last_four_digits_installments_plans,card_first_six_digits_installments_plans,is_apple_pay_installments_plans,payment_method_country_installments_plans,payment_method_bank_installments_plans,payment_method_type_installments_plans,customer_email_installments_plans,payment_method_fingerprint_installments_plans,...,email_details_domain_details_tld_seon,email_details_domain_details_disposable_seon,email_details_domain_details_spf_strict_seon,email_details_domain_details_website_exists_seon,email_details_domain_details_domain_seon,email_details_domain_details_dmarc_enforced_seon,email_details_breach_details_first_breach_seon,email_details_breach_details_haveibeenpwned_listed_seon,email_details_breach_details_number_of_breaches_seon,fraud_category_fraudcategory


In [ ]:
# orders_agg_cleaned.loc[orders_agg_cleaned['customer_date_joined_orders']=='1754-08-28 22:43:41.128654848', 'customer_date_joined_orders'] = np.nan
# orders_agg_cleaned.loc[orders_agg_cleaned['customer_date_joined_orders']=='1900-01-01 09:28:14.790000', 'customer_date_joined_orders'] = np.nan
# orders_agg_cleaned.loc[orders_agg_cleaned['date_of_birth_installments_plans']=='1682-09-19 00:00:00', 'date_of_birth_installments_plans'] = np.nan
# orders_agg_cleaned.loc[orders_agg_cleaned['date_of_birth_installments_plans']=='1683-06-05 00:00:00', 'date_of_birth_installments_plans'] = np.nan
# orders_agg_cleaned.loc[orders_agg_cleaned['date_of_birth_installments_plans']=='1684-05-09 00:00:00', 'date_of_birth_installments_plans'] = np.nan


In [48]:
# datetime conversion
orders_agg_cleaned['created_orders'] = pd.to_datetime(orders_agg_cleaned['created_orders'], errors='coerce')
orders_agg_cleaned['customer_date_joined_orders'] = pd.to_datetime(orders_agg_cleaned['customer_date_joined_orders'], errors='coerce')
orders_agg_cleaned['checkout_verified_orders'] = pd.to_datetime(orders_agg_cleaned['checkout_verified_orders'], errors='coerce')
orders_agg_cleaned['payment_method_expires_orders'] = pd.to_datetime(orders_agg_cleaned['payment_method_expires_orders'], errors='coerce')
orders_agg_cleaned['payment_method_expires_installments_plans'] = pd.to_datetime(orders_agg_cleaned['payment_method_expires_installments_plans'], errors='coerce')
orders_agg_cleaned['date_of_birth_installments_plans'] = pd.to_datetime(orders_agg_cleaned['date_of_birth_installments_plans'], errors='coerce')
orders_agg_cleaned['date_of_birth_customers'] = pd.to_datetime(orders_agg_cleaned['date_of_birth_customers'], errors='coerce')
orders_agg_cleaned['date_of_birth_formatted_idv'] = pd.to_datetime(orders_agg_cleaned['date_of_birth_formatted_idv'], errors='coerce')
orders_agg_cleaned['date_of_expiry_formatted_idv'] = pd.to_datetime(orders_agg_cleaned['date_of_expiry_formatted_idv'], errors='coerce')
orders_agg_cleaned['primary_email_domain_creation_date_ekata'] = pd.to_datetime(orders_agg_cleaned['primary_email_domain_creation_date_ekata'], errors='coerce')
orders_agg_cleaned['email_details_domain_details_created_seon'] = pd.to_datetime(orders_agg_cleaned['email_details_domain_details_created_seon'], errors='coerce')
orders_agg_cleaned['email_details_breach_details_first_breach_seon'] = pd.to_datetime(orders_agg_cleaned['email_details_breach_details_first_breach_seon'], errors='coerce')

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/200276026.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned['created_orders'] = pd.to_datetime(orders_agg_cleaned['created_orders'], errors='coerce')
/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/200276026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned['customer_date_joined_orders'] = pd.to_datetime(orders_agg_cleaned['customer_date_joined_orders'], errors='coerce')
/var/folders/3

In [49]:
# convert to unix timestamp
import time

orders_agg_cleaned['TransactionDT'] = orders_agg_cleaned['created_orders'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['customer_date_joined'] = orders_agg_cleaned['customer_date_joined_orders'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['checkout_verified'] = orders_agg_cleaned['checkout_verified_orders'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['payment_method_expires_orders'] = orders_agg_cleaned['payment_method_expires_orders'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['payment_method_expires_installments_plans'] = orders_agg_cleaned['payment_method_expires_installments_plans'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)

orders_agg_cleaned['date_of_birth_installments_plans'] = orders_agg_cleaned['date_of_birth_installments_plans'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['date_of_birth_customers'] = orders_agg_cleaned['date_of_birth_customers'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['date_of_birth_formatted_idv'] = orders_agg_cleaned['date_of_birth_formatted_idv'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['date_of_expiry_formatted_idv'] = orders_agg_cleaned['date_of_expiry_formatted_idv'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['primary_email_domain_creation_date_ekata'] = orders_agg_cleaned['primary_email_domain_creation_date_ekata'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['email_details_domain_details_created_seon'] = orders_agg_cleaned['email_details_domain_details_created_seon'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
orders_agg_cleaned['email_details_breach_details_first_breach_seon'] = orders_agg_cleaned['email_details_breach_details_first_breach_seon'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/600377930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned['TransactionDT'] = orders_agg_cleaned['created_orders'].apply(lambda x: x.timestamp() if not pd.isnull(x) else np.nan)
/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/600377930.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned['customer_date_joined'] = orders_agg_cleaned['customer_date_joined_orders'].apply(lambda x: x.timesta

In [50]:
# fill missing date values
orders_agg_cleaned['customer_date_joined'].fillna(orders_agg_cleaned['customer_date_joined'].median(), inplace=True)
orders_agg_cleaned['checkout_verified'].fillna(orders_agg_cleaned['checkout_verified'].median(), inplace=True)

orders_agg_cleaned['payment_method_expires'] = orders_agg_cleaned['payment_method_expires_orders'].fillna(orders_agg_cleaned['payment_method_expires_installments_plans'])
orders_agg_cleaned['payment_method_expires'].fillna(orders_agg_cleaned['payment_method_expires'].median(), inplace=True)

orders_agg_cleaned['date_of_birth_cleaned'] = orders_agg_cleaned['date_of_birth_formatted_idv'].fillna(orders_agg_cleaned['date_of_birth_installments_plans']).fillna(orders_agg_cleaned['date_of_birth_customers'])
orders_agg_cleaned['date_of_birth_cleaned'].fillna(orders_agg_cleaned['date_of_birth_cleaned'].median(), inplace=True)

orders_agg_cleaned['date_of_expiry_formatted_idv'].fillna(orders_agg_cleaned['date_of_expiry_formatted_idv'].median(), inplace=True)
orders_agg_cleaned['primary_email_domain_creation_date_ekata'].fillna(orders_agg_cleaned['primary_email_domain_creation_date_ekata'].median(), inplace=True)
orders_agg_cleaned['email_details_domain_details_created_seon'].fillna(orders_agg_cleaned['email_details_domain_details_created_seon'].median(), inplace=True)
orders_agg_cleaned['email_details_breach_details_first_breach_seon'].fillna(orders_agg_cleaned['email_details_breach_details_first_breach_seon'].median(), inplace=True)

orders_agg_cleaned['card_brand_cleaned_bin'] = orders_agg_cleaned['card_brand_cleaned_bin'].fillna(orders_agg_cleaned['payment_method_brand_installments_plans'])
orders_agg_cleaned['card_last_four_digits_orders'] = orders_agg_cleaned['card_last_four_digits_orders'].fillna(orders_agg_cleaned['card_last_four_digits_installments_plans'])
orders_agg_cleaned['card_first_six_digits_orders'] = orders_agg_cleaned['card_first_six_digits_orders'].fillna(orders_agg_cleaned['card_first_six_digits_installments_plans'])
orders_agg_cleaned['payment_method_country_orders'] = orders_agg_cleaned['payment_method_country_orders'].fillna(orders_agg_cleaned['payment_method_country_installments_plans'])
orders_agg_cleaned['payment_method_bank_orders'] = orders_agg_cleaned['payment_method_bank_orders'].fillna(orders_agg_cleaned['payment_method_bank_installments_plans'])
orders_agg_cleaned['card_type_cleaned_bin'] = orders_agg_cleaned['card_type_cleaned_bin'].fillna(orders_agg_cleaned['payment_method_type_installments_plans'])
orders_agg_cleaned['is_apple_pay_orders'] = orders_agg_cleaned['is_apple_pay_orders'].fillna(orders_agg_cleaned['is_apple_pay_installments_plans'])
# orders_agg_cleaned['secure_orders'] = orders_agg_cleaned['secure_orders'].fillna(orders_agg_cleaned['secure_installments_plans'])
orders_agg_cleaned['payment_method_fingerprint_orders'] = orders_agg_cleaned['payment_method_fingerprint_orders'].fillna(orders_agg_cleaned['payment_method_fingerprint_installments_plans'])

orders_agg_cleaned['customer_email_orders'] = orders_agg_cleaned['customer_email_orders'].fillna(orders_agg_cleaned['customer_email_installments_plans']).fillna(orders_agg_cleaned['email_customers'])
orders_agg_cleaned['phone_installments_plans'] = orders_agg_cleaned['phone_installments_plans'].fillna(orders_agg_cleaned['phone_customers'])

orders_agg_cleaned['full_name_customers'] = (orders_agg_cleaned['first_name_customers'].fillna('') + ' ' + orders_agg_cleaned['last_name_customers'].fillna(''))
orders_agg_cleaned['full_name_addresses'] = (orders_agg_cleaned['first_name_addresses'].fillna('') + ' ' + orders_agg_cleaned['last_name_addresses'].fillna(''))

orders_agg_cleaned['full_address_addresses'] = (orders_agg_cleaned['line1_addresses'].fillna('') + ' ' +
                                                orders_agg_cleaned['line2_addresses'].fillna(''))
orders_agg_cleaned['final_address_district'] = orders_agg_cleaned['final_address_district'].fillna(orders_agg_cleaned['full_address_addresses'])

orders_agg_cleaned['id_number_installments_plans'] = orders_agg_cleaned['id_number_installments_plans'].fillna(orders_agg_cleaned['id_number_customers'])




/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/3372636735.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders_agg_cleaned['customer_date_joined'].fillna(orders_agg_cleaned['customer_date_joined'].median(), inplace=True)
/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/3372636735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [51]:
# calculating card expiry in months
from datetime import datetime

current_date = pd.Timestamp(datetime.now())
orders_agg_cleaned['payment_method_expires_in_months'] = ((pd.to_datetime(orders_agg_cleaned['payment_method_expires'], unit='s') - pd.to_datetime(orders_agg_cleaned['TransactionDT'], unit='s')) / pd.Timedelta(days=30)).astype(int)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/3008081364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned['payment_method_expires_in_months'] = ((pd.to_datetime(orders_agg_cleaned['payment_method_expires'], unit='s') - pd.to_datetime(orders_agg_cleaned['TransactionDT'], unit='s')) / pd.Timedelta(days=30)).astype(int)


In [52]:
# adding card expires before tenure flag
orders_agg_cleaned['card_expires_before_tenure'] = np.where(orders_agg_cleaned['num_instalments_orders'] >= orders_agg_cleaned['payment_method_expires_in_months'], True, False)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/816759196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned['card_expires_before_tenure'] = np.where(orders_agg_cleaned['num_instalments_orders'] >= orders_agg_cleaned['payment_method_expires_in_months'], True, False)


In [53]:
# drop unnecessary columns
orders_agg_cleaned.drop(columns=['created_orders', 'customer_date_joined_orders', 'checkout_verified_orders', 'payment_method_expires_orders', 
                                 'date_of_birth_installments_plans', 'date_of_birth_customers', 'date_of_birth_formatted_idv', 'checkout_metadata_orders', 
                                 'payment_method_brand_installments_plans', 'payment_method_expires_installments_plans', 'card_last_four_digits_installments_plans', 'card_first_six_digits_installments_plans', 'is_apple_pay_installments_plans',
                                 'payment_method_country_installments_plans', 'payment_method_bank_installments_plans', 'payment_method_type_installments_plans', 'customer_email_installments_plans', 
                                 # 'secure_installments_plans',
                                 'payment_method_fingerprint_installments_plans', 'phone_customers', 'phone_addresses', 'first_name_customers', 'last_name_customers', 'first_name_addresses', 'last_name_addresses', 
                                 'line1_addresses', 'line2_addresses', 'full_address_addresses', 'id_number_customers', 'email_customers'], 
                        inplace=True)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/1705524371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned.drop(columns=['created_orders', 'customer_date_joined_orders', 'checkout_verified_orders', 'payment_method_expires_orders',


In [54]:
# adding new derived columns
# orders_agg_cleaned['card_first_six_digits_orders'] = orders_agg_cleaned['card_first_six_digits_orders'].astype('int64')
# orders_agg_cleaned['card_last_four_digits_orders'] = orders_agg_cleaned['card_last_four_digits_orders'].astype('int64')

orders_agg_cleaned['card_unique_identifier'] = orders_agg_cleaned['card_first_six_digits_orders'].astype(str) + '***' + orders_agg_cleaned['card_last_four_digits_orders'].astype(str)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/2595032985.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned['card_unique_identifier'] = orders_agg_cleaned['card_first_six_digits_orders'].astype(str) + '***' + orders_agg_cleaned['card_last_four_digits_orders'].astype(str)


In [55]:
orders_agg_cleaned.drop(columns=['card_last_four_digits_orders', 'card_first_six_digits_orders'],
                        inplace=True)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/96225648.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_agg_cleaned.drop(columns=['card_last_four_digits_orders', 'card_first_six_digits_orders'],


In [56]:
# filter desired period
orders_agg_cleaned = orders_agg_cleaned.loc[(orders_agg_cleaned['TransactionDT'] >= pd.Timestamp('2022-01-01 00:00:00').timestamp()) & (orders_agg_cleaned['TransactionDT'] <= pd.Timestamp('2023-10-31 00:00:00').timestamp())]

In [63]:
# filling missing values in numerical columns
orders_agg_cleaned[orders_agg_cleaned['phone_details_score_seon'].isna()==True].head()

,id_number_installments_plans,order_id_orders,total_amount_orders,customer_email_orders,merchant_name_orders,category_merchantcategory,checkout_type_orders,num_instalments_orders,ip_address_orders,user_agent_orders,...,TransactionDT,customer_date_joined,checkout_verified,payment_method_expires,date_of_birth_cleaned,full_name_customers,full_name_addresses,payment_method_expires_in_months,card_expires_before_tenure,card_unique_identifier


In [59]:
orders_agg_cleaned[orders_agg_cleaned['card_unique_identifier'].isna()==True]#.sum()

,id_number_installments_plans,order_id_orders,total_amount_orders,customer_email_orders,merchant_name_orders,category_merchantcategory,checkout_type_orders,num_instalments_orders,ip_address_orders,user_agent_orders,...,TransactionDT,customer_date_joined,checkout_verified,payment_method_expires,date_of_birth_cleaned,full_name_customers,full_name_addresses,payment_method_expires_in_months,card_expires_before_tenure,card_unique_identifier


In [65]:
# orders[orders['order_id_orders']==1712883]

In [61]:
# filling missing values in numerical columns
orders_agg_cleaned['total_quantity'].fillna(0, inplace=True)
orders_agg_cleaned['max_quantity'].fillna(0, inplace=True)

orders_agg_cleaned['ip_last_seen_days_ekata'].fillna(orders_agg_cleaned['ip_last_seen_days_ekata'].median(), inplace=True)
orders_agg_cleaned['ip_risk_score_ekata'].fillna(orders_agg_cleaned['ip_risk_score_ekata'].median(), inplace=True)
orders_agg_cleaned['primary_email_last_seen_days_ekata'].fillna(orders_agg_cleaned['primary_email_last_seen_days_ekata'].median(), inplace=True)
orders_agg_cleaned['ip_primary_address_distance_ekata'].fillna(orders_agg_cleaned['ip_primary_address_distance_ekata'].median(), inplace=True)
orders_agg_cleaned['identity_network_score_ekata'].fillna(orders_agg_cleaned['identity_network_score_ekata'].median(), inplace=True)
orders_agg_cleaned['identity_risk_score_ekata'].fillna(orders_agg_cleaned['identity_risk_score_ekata'].median(), inplace=True)
orders_agg_cleaned['primary_address_first_seen_days_ekata'].fillna(orders_agg_cleaned['primary_address_first_seen_days_ekata'].median(), inplace=True)
orders_agg_cleaned['primary_phone_ip_last_seen_days_ekata'].fillna(orders_agg_cleaned['primary_phone_ip_last_seen_days_ekata'].median(), inplace=True)
orders_agg_cleaned['primary_email_first_seen_days_ekata'].fillna(orders_agg_cleaned['primary_email_first_seen_days_ekata'].median(), inplace=True)
orders_agg_cleaned['phone_details_score_seon'].fillna(orders_agg_cleaned['phone_details_score_seon'].median(), inplace=True)
orders_agg_cleaned['ip_details_spam_number_seon'].fillna(orders_agg_cleaned['ip_details_spam_number_seon'].median(), inplace=True)
orders_agg_cleaned['ip_details_score_seon'].fillna(orders_agg_cleaned['ip_details_score_seon'].median(), inplace=True)
orders_agg_cleaned['fraud_score_seon'].fillna(orders_agg_cleaned['fraud_score_seon'].median(), inplace=True)
orders_agg_cleaned['email_details_score_seon'].fillna(orders_agg_cleaned['email_details_score_seon'].median(), inplace=True)
orders_agg_cleaned['email_details_breach_details_number_of_breaches_seon'].fillna(orders_agg_cleaned['email_details_breach_details_number_of_breaches_seon'].median(), inplace=True)
orders_agg_cleaned['payment_method_expires_in_months'].fillna(orders_agg_cleaned['payment_method_expires_in_months'].median(), inplace=True)

# filling missing values in categorical columns
orders_agg_cleaned['customer_email_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['category_merchantcategory'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_address_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['user_agent_orders'].fillna('unknown', inplace=True)
# orders_agg_cleaned['secure_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['device_fingerprint_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['payment_method_fingerprint_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['id_number_installments_plans'].fillna('unknown', inplace=True)
orders_agg_cleaned['is_apple_pay_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['merchant_name_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['payment_method_country_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['checkout_type_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['num_instalments_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['payment_method_bank_orders'].fillna('unknown', inplace=True)
orders_agg_cleaned['card_type_cleaned_bin'].fillna('unknown', inplace=True)
orders_agg_cleaned['card_brand_cleaned_bin'].fillna('unknown', inplace=True)
orders_agg_cleaned['card_level_cleaned_bin'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_installments_plans'].fillna('unknown', inplace=True)
orders_agg_cleaned['final_address_district'].fillna('unknown', inplace=True)
orders_agg_cleaned['formatted_address_district'].fillna('unknown', inplace=True)
orders_agg_cleaned['state_emirate_district'].fillna('unknown', inplace=True)
orders_agg_cleaned['district_district'].fillna('unknown', inplace=True)
orders_agg_cleaned['country_district'].fillna('unknown', inplace=True)
orders_agg_cleaned['concatenated_names'].fillna('unknown', inplace=True)
orders_agg_cleaned['max_quantity_name'].fillna('unknown', inplace=True)
orders_agg_cleaned['full_name_idv'].fillna('unknown', inplace=True)
orders_agg_cleaned['identity_number_idv'].fillna('unknown', inplace=True)
orders_agg_cleaned['place_of_issue_idv'].fillna('unknown', inplace=True)
orders_agg_cleaned['sex_idv'].fillna('unknown', inplace=True)
orders_agg_cleaned['employer_idv'].fillna('unknown', inplace=True)
orders_agg_cleaned['occupation_group_idv'].fillna('unknown', inplace=True)
orders_agg_cleaned['nationality_code_idv'].fillna('unknown', inplace=True)
orders_agg_cleaned['primary_email_to_name_ekata'].fillna('unknown', inplace=True)
# orders_agg_cleaned['secondary_address_validity_level_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_geolocation_country_code_ekata'].fillna('unknown', inplace=True)
# orders_agg_cleaned['primary_phone_valid_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['primary_address_validity_level_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_risk_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['primary_email_valid_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['primary_phone_to_name_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['primary_phone_carrier_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['primary_phone_line_type_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_connection_type_ekata'].fillna('unknown', inplace=True)
orders_agg_cleaned['primary_address_to_name_ekata'].fillna('unknown', inplace=True)
# orders_agg_cleaned['phone_details_valid_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_country_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_carrier_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_account_details_whatsapp_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_account_details_twitter_registered_seon'].fillna('unknown', inplace=True)
# orders_agg_cleaned['phone_details_account_details_yahoo_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_account_details_facebook_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_account_details_telegram_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_account_details_instagram_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['phone_details_account_details_google_registered_seon'].fillna('unknown', inplace=True)
# orders_agg_cleaned['phone_details_type_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['state_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_details_country_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_details_public_proxy_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_details_city_seon'].fillna('unknown', inplace=True)
# orders_agg_cleaned['ip_details_web_proxy_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_details_type_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_details_isp_name_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_details_vpn_seon'].fillna('unknown', inplace=True)
# orders_agg_cleaned['ip_details_tor_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['ip_details_state_prov_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_github_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_facebook_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_spotify_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_google_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_pinterest_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_instagram_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_linkedin_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_microsoft_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_apple_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_twitter_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_deliverable_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_suspicious_tld_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_custom_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_valid_mx_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_accept_all_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_registrar_name_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_registered_to_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_tld_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_disposable_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_spf_strict_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_website_exists_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_domain_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_domain_details_dmarc_enforced_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_account_details_linkedin_registered_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['email_details_breach_details_haveibeenpwned_listed_seon'].fillna('unknown', inplace=True)
orders_agg_cleaned['full_name_customers'].fillna('unknown', inplace=True)
orders_agg_cleaned['full_name_addresses'].fillna('unknown', inplace=True)
orders_agg_cleaned['card_expires_before_tenure'].fillna('unknown', inplace=True)
orders_agg_cleaned['card_unique_identifier'].fillna('unknown', inplace=True)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/2011538286.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders_agg_cleaned['total_quantity'].fillna(0, inplace=True)
/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/2011538286.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [66]:
# target variable
orders_agg_cleaned['isFraud'] = np.where(orders_agg_cleaned['fraud_category_fraudcategory'].isin(['Fraud', 'Suspected Fraud']), True, False)

In [67]:
orders_agg_cleaned = orders_agg_cleaned[~orders_agg_cleaned['fraud_category_fraudcategory'].isna()].reset_index(drop=True)

In [68]:
orders_agg_cleaned[['isFraud', 'fraud_category_fraudcategory']].value_counts()

isFraud  fraud_category_fraudcategory
False    Legit                           277387
         Grey Area                         6092
True     Fraud                             5927
         Suspected Fraud                   1979
Name: count, dtype: int64

In [69]:
orders_agg_cleaned.drop(columns=['fraud_category_fraudcategory'], inplace=True)

In [70]:
orders_agg_cleaned.drop(columns=['email_details_account_details_linkedin_registered_seon'], inplace=True)

In [77]:
orders_agg_cleaned[orders_agg_cleaned.isna().any(axis=1)].sum()

id_number_installments_plans          0
order_id_orders                       0
total_amount_orders                 0.0
customer_email_orders                 0
merchant_name_orders                  0
                                   ... 
full_name_addresses                   0
payment_method_expires_in_months      0
card_expires_before_tenure            0
card_unique_identifier                0
isFraud                               0
Length: 115, dtype: object

In [78]:
orders_agg_cleaned.columns[orders_agg_cleaned.isna().any()].tolist()

[]

In [79]:
orders_agg_cleaned[orders_agg_cleaned.isna().any(axis=1)]

,id_number_installments_plans,order_id_orders,total_amount_orders,customer_email_orders,merchant_name_orders,category_merchantcategory,checkout_type_orders,num_instalments_orders,ip_address_orders,user_agent_orders,...,customer_date_joined,checkout_verified,payment_method_expires,date_of_birth_cleaned,full_name_customers,full_name_addresses,payment_method_expires_in_months,card_expires_before_tenure,card_unique_identifier,isFraud


#### Custom Feature Engineering

In [81]:
# new features
orders_features = pd.read_csv(rawData_dir / 'orders.csv')

orders_features = orders_features.loc[(orders_features['num_instalments'] >1) &
           (orders_features['currency'] == 'AED') &
           (orders_features['status'].isin(['captured', 'denied', 'pending']))
           ]

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/1477044316.py:2: DtypeWarning: Columns (2,25,26,27,28,29,35,37,39,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  orders_features = pd.read_csv(rawData_dir / 'orders.csv')


In [82]:
orders_features = pd.merge(orders_features, installments_plans, how='left', left_on='order_id', right_on='order_id_installments_plans')

In [83]:
orders_features['card_last_four_digits'] = orders_features['card_last_four_digits'].fillna(orders_features['card_last_four_digits_installments_plans'])
orders_features['card_first_six_digits'] = orders_features['card_first_six_digits'].fillna(orders_features['card_first_six_digits_installments_plans'])

orders_features['customer_email'] = orders_features['customer_email'].fillna(orders_features['customer_email_installments_plans'])

In [84]:
orders_features['card_unique_identifier'] = orders_features['card_first_six_digits'].astype(str) + '***' + orders_features['card_last_four_digits'].astype(str)

In [85]:
orders_features[orders_features['card_unique_identifier'].isna()==True]

,order_id,shipping_id,merchant_id,total_amount,status,status_changed,billing_address_id,convenience_fee,tax_amount,created,...,user_agent_installments_plans,downpayment_refunded_amount_installments_plans,ip_address_installments_plans,transaction_cost_rate_installments_plans,transaction_cost_amount_installments_plans,total_amount_unpaid_installments_plans,total_amount_due_installments_plans,total_amount_paid_installments_plans,instalment_plan_status_installments_plans,card_unique_identifier


In [86]:
grouped_counts = orders_features.groupby(['card_unique_identifier', 'status']).size().reset_index(name='counts')
pivot = grouped_counts.pivot(index='card_unique_identifier', columns='status', values='counts').fillna(0)
pivot = pivot.reset_index()
pivot.columns = ['card_unique_identifier', 'captured', 'denied', 'pending']
card_orders_count = pivot

In [87]:
cards_due_unpaid = orders_features.groupby(['card_unique_identifier']).agg(
    due=('total_amount_due_installments_plans', 'sum'),
    unpaid=('total_amount_unpaid_installments_plans', 'sum'),
    phones=('phone_installments_plans', 'nunique'),
    emails=('customer_email', 'nunique')
).reset_index()

In [88]:
cards_due_unpaid['total_due_unpaid'] = cards_due_unpaid['due'] + cards_due_unpaid['unpaid']
card_orders_count['total_orders'] = card_orders_count['captured'] + card_orders_count['denied'] + card_orders_count['pending']

In [89]:
card_orders_exposure = pd.merge(card_orders_count, cards_due_unpaid, how='left', on='card_unique_identifier')

In [124]:
# card_orders_exposure

In [91]:
# card_orders_exposure[card_orders_exposure['pending']==460992.0]

In [92]:
# TODO: add days between latest trx on card (overall and captured only)
# TODO: add days between latest trx on customer id (overall and captured only)

In [93]:
orders_features['created'] = pd.to_datetime(orders_features['created'])

# Sort the DataFrame by card_number and transaction_date
orders_features = orders_features.sort_values(by=['card_unique_identifier', 'created'])

# Use groupby and shift to create a column for the last transaction date per card
orders_features['previous_transaction_date'] = orders_features.groupby('card_unique_identifier')['created'].shift(1)

# Calculate the difference in days between the current and previous transaction dates
orders_features['days_since_last_transaction'] = (orders_features['created'] - orders_features['previous_transaction_date']).dt.total_seconds() / (60 * 60 * 24)

In [94]:
card_last_application_days = orders_features[['order_id','days_since_last_transaction']]

In [95]:
# card_last_application_days['days_since_last_transaction'].fillna(card_last_application_days['days_since_last_transaction'].median(), inplace=True)

In [125]:
# card_last_application_days

In [97]:
orders_agg_cleaned = pd.merge(orders_agg_cleaned, card_last_application_days, how='left', left_on='order_id_orders', right_on='order_id')

In [98]:
orders_agg_cleaned.drop(columns=['order_id'], inplace=True)

In [99]:
orders_agg_cleaned = pd.merge(orders_agg_cleaned, card_orders_exposure, how='left', on='card_unique_identifier')

In [100]:
orders_agg_cleaned['days_since_last_transaction'].fillna(orders_agg_cleaned['days_since_last_transaction'].median(), inplace=True)

/var/folders/3s/rv8h_gcs26l2ll42x85k40j80000gn/T/ipykernel_20911/2050368766.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  orders_agg_cleaned['days_since_last_transaction'].fillna(orders_agg_cleaned['days_since_last_transaction'].median(), inplace=True)


In [104]:
orders_agg_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291385 entries, 0 to 291384
Columns: 125 entries, id_number_installments_plans to total_due_unpaid
dtypes: bool(2), float64(36), int64(5), object(82)
memory usage: 274.0+ MB


In [102]:
# rename columns
orders_agg_cleaned.rename(columns={'total_amount_orders': 'TransactionAmt', 'order_id_orders': 'TransactionID'}, inplace=True)

In [105]:
orders_agg_cleaned = orders_agg_cleaned.replace(',', ' ', regex=True)

In [106]:
import re

def remove_special_characters(text):
    pattern = r'[^a-zA-Z0-9\s]'
    pattern_with_newline = r'[\n]'
    
    text = re.sub(pattern, '', text)
    text = re.sub(pattern_with_newline, ' ', text)
    return text.strip()

orders_agg_cleaned['user_agent_orders'] = orders_agg_cleaned['user_agent_orders'].apply(remove_special_characters)
orders_agg_cleaned['final_address_district'] = orders_agg_cleaned['final_address_district'].apply(remove_special_characters)
orders_agg_cleaned['formatted_address_district'] = orders_agg_cleaned['formatted_address_district'].apply(remove_special_characters)
orders_agg_cleaned['concatenated_names'] = orders_agg_cleaned['concatenated_names'].apply(remove_special_characters)
orders_agg_cleaned['max_quantity_name'] = orders_agg_cleaned['max_quantity_name'].apply(remove_special_characters)

In [107]:
def remove_special_characters_add_underscore(text):
    pattern = r'[^a-zA-Z0-9\s]'
    pattern_with_newline = r'[\n]'

    text = re.sub(pattern, '', text)
    text = re.sub(pattern_with_newline, ' ', text)
    text = '_'.join(text.split())
    return text.strip()

In [108]:
orders_agg_cleaned['employer_idv'] = orders_agg_cleaned['employer_idv'].apply(remove_special_characters_add_underscore)
orders_agg_cleaned['occupation_group_idv'] = orders_agg_cleaned['occupation_group_idv'].apply(remove_special_characters_add_underscore)
orders_agg_cleaned['category_merchantcategory'] = orders_agg_cleaned['category_merchantcategory'].apply(remove_special_characters_add_underscore)

orders_agg_cleaned['place_of_issue_idv'] = orders_agg_cleaned['place_of_issue_idv'].apply(remove_special_characters_add_underscore)

orders_agg_cleaned['user_agent_orders'] = orders_agg_cleaned['user_agent_orders'].apply(remove_special_characters_add_underscore)
orders_agg_cleaned['final_address_district'] = orders_agg_cleaned['final_address_district'].apply(remove_special_characters_add_underscore)
orders_agg_cleaned['formatted_address_district'] = orders_agg_cleaned['formatted_address_district'].apply(remove_special_characters_add_underscore)
orders_agg_cleaned['concatenated_names'] = orders_agg_cleaned['concatenated_names'].apply(remove_special_characters_add_underscore)
orders_agg_cleaned['max_quantity_name'] = orders_agg_cleaned['max_quantity_name'].apply(remove_special_characters_add_underscore)


In [114]:
# Temporal splits
full_dataset = orders_agg_cleaned.copy()
dataset_train = orders_agg_cleaned.loc[~(orders_agg_cleaned['TransactionDT'] >= pd.Timestamp('2023-07-01 00:00:00').timestamp())]
dataset_dev = orders_agg_cleaned.loc[(orders_agg_cleaned['TransactionDT'] >= pd.Timestamp('2023-07-01 00:00:00').timestamp()) & (orders_agg_cleaned['TransactionDT'] <= pd.Timestamp('2023-07-31 00:00:00').timestamp())]
dataset_test = orders_agg_cleaned.loc[(orders_agg_cleaned['TransactionDT'] > pd.Timestamp('2023-07-31 00:00:00').timestamp())]

In [115]:
full_dataset['isFraud'].value_counts()

isFraud
False    283479
True       7906
Name: count, dtype: int64

In [116]:
dataset_train['isFraud'].value_counts()

isFraud
False    245767
True       5261
Name: count, dtype: int64

In [117]:
dataset_test['isFraud'].value_counts()

isFraud
False    26462
True      2180
Name: count, dtype: int64

In [118]:
dataset_dev['isFraud'].value_counts()

isFraud
False    11250
True       465
Name: count, dtype: int64

In [119]:
# split into transaction data and identity data
transactions_cols = ['TransactionID', 'TransactionAmt', 'category_merchantcategory', 'checkout_type_orders', 'num_instalments_orders', 
                     # 'secure_orders', 
                     'is_apple_pay_orders', 'total_quantity', 'max_quantity', 'date_of_expiry_formatted_idv', 'place_of_issue_idv', 
                     'sex_idv', 'employer_idv', 'occupation_group_idv', 'nationality_code_idv', 
                     'TransactionDT',
                     'customer_date_joined',
                     'checkout_verified',
                     'payment_method_expires',
                     'date_of_birth_cleaned',
                     'payment_method_expires_in_months',
                     'card_expires_before_tenure',

                     'fraud_score_seon',
                     'email_details_score_seon',
                     
                     'email_details_domain_details_created_seon',
                     'email_details_breach_details_number_of_breaches_seon',
                     
                     'email_details_domain_details_suspicious_tld_seon',
                     'email_details_domain_details_custom_seon',
                     'email_details_domain_details_valid_mx_seon',
                     'email_details_domain_details_accept_all_seon',
                     'email_details_domain_details_registered_seon',
                     'email_details_domain_details_disposable_seon',
                     'email_details_domain_details_spf_strict_seon',
                     'email_details_domain_details_website_exists_seon',
                     'email_details_domain_details_dmarc_enforced_seon',
                     
                     'ip_last_seen_days_ekata',
                     'primary_email_to_name_ekata',
                     'primary_email_first_seen_days_ekata',
                     'state_seon',
                     'ip_details_score_seon',
                     'primary_phone_ip_last_seen_days_ekata',
                     'phone_details_score_seon',
                     'ip_details_spam_number_seon',
                     'primary_email_last_seen_days_ekata',
                     'identity_risk_score_ekata',
                     'primary_email_domain_creation_date_ekata',
                     'ip_risk_score_ekata',
                     'ip_primary_address_distance_ekata',
                     'identity_network_score_ekata',
                     'primary_address_first_seen_days_ekata',
                     
                     'isFraud',
                     
                     'days_since_last_transaction',
                        'captured', 'denied', 'pending', 'total_orders', 'due', 'unpaid', 'phones', 'emails', 'total_due_unpaid'
                     ]

identity_cols = ['TransactionID', 'id_number_installments_plans', 'customer_email_orders', 'merchant_name_orders', 'ip_address_orders', 
                 'user_agent_orders', 'device_fingerprint_orders', 'payment_method_fingerprint_orders', 'payment_method_country_orders', 
                 'payment_method_bank_orders', 'card_type_cleaned_bin', 'card_brand_cleaned_bin', 'card_level_cleaned_bin', 
                 'phone_installments_plans', 'final_address_district', 'formatted_address_district', 'state_emirate_district', 
                 'district_district', 'country_district', 'concatenated_names', 'max_quantity_name', 'full_name_idv', 'identity_number_idv',
                 'card_unique_identifier',
                 'full_name_customers',
                 'full_name_addresses',

                 'ip_geolocation_country_code_ekata',
                 'email_details_domain_details_registrar_name_seon',
                 'email_details_domain_details_registered_to_seon',
                 'email_details_domain_details_tld_seon',
                 'email_details_domain_details_domain_seon',
                 'primary_phone_carrier_ekata',
                 'primary_phone_line_type_ekata',
                 'ip_connection_type_ekata',
                 
                 'phone_details_country_seon',
                 'phone_details_carrier_seon',
                 # 'phone_details_type_seon',
                 'ip_details_country_seon',
                 'ip_details_city_seon',
                 'ip_details_type_seon',
                 'ip_details_isp_name_seon',
                 'ip_details_state_prov_seon',
                 'ip_details_vpn_seon',
                 # 'ip_details_tor_seon',
                 'email_details_deliverable_seon',
                 'email_details_account_details_github_registered_seon',
                 'email_details_account_details_facebook_registered_seon',
                 'email_details_account_details_spotify_registered_seon',
                 'email_details_account_details_google_registered_seon',
                 'email_details_account_details_pinterest_registered_seon',
                 'email_details_account_details_instagram_registered_seon',
                 'email_details_account_details_microsoft_registered_seon',
                 'email_details_account_details_apple_registered_seon',
                 'email_details_account_details_twitter_registered_seon',
                 
                 # 'secondary_address_validity_level_ekata', 
                 # 'primary_phone_valid_ekata', 
                 'primary_address_validity_level_ekata', 
                 'ip_risk_ekata',   'primary_email_valid_ekata', 'primary_phone_to_name_ekata',
                    
                  'primary_address_to_name_ekata',  
                 # 'phone_details_valid_seon',
                 
                  'phone_details_account_details_whatsapp_registered_seon',
                 'phone_details_account_details_twitter_registered_seon',
                 # 'phone_details_account_details_yahoo_registered_seon',
                 'phone_details_account_details_facebook_registered_seon',
                 'phone_details_account_details_telegram_registered_seon',
                 'phone_details_account_details_instagram_registered_seon',
                 'phone_details_account_details_google_registered_seon', 'ip_details_public_proxy_seon', 
                 # 'ip_details_web_proxy_seon',
                  

                 
                 
                 
                 
                 
                 
                 'email_details_breach_details_first_breach_seon',
                 'email_details_breach_details_haveibeenpwned_listed_seon',
                 
                 
                 ]


In [ ]:
preprocessedData_dir = cwd / "preprocessedData"

In [120]:
full_transaction = full_dataset[transactions_cols]
full_identity = full_dataset[identity_cols]

train_transaction = dataset_train[transactions_cols]
train_identity = dataset_train[identity_cols]

dev_transaction = dataset_dev[transactions_cols]
dev_identity = dataset_dev[identity_cols]

test_transaction = dataset_test[transactions_cols]
test_identity = dataset_test[identity_cols]

In [121]:
full_transaction.to_csv(preprocessedData_dir / 'full_transaction.csv', index=False)
full_identity.to_csv(preprocessedData_dir / 'full_identity.csv', index=False)

train_transaction.to_csv(preprocessedData_dir / 'train_transaction.csv', index=False)
train_identity.to_csv(preprocessedData_dir / 'train_identity.csv', index=False)

dev_transaction.to_csv(preprocessedData_dir / 'dev_transaction.csv', index=False)
dev_identity.to_csv(preprocessedData_dir / 'dev_identity.csv', index=False)

test_transaction.to_csv(preprocessedData_dir / 'test_transaction.csv', index=False)
test_identity.to_csv(preprocessedData_dir / 'test_identity.csv', index=False)

In [123]:
# train_transaction.describe(include='all')